In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from sklearn.metrics import confusion_matrix, mean_squared_error, log_loss, accuracy_score
import time
from tensorflow.keras.models import model_from_json
from tensorflow.keras import layers
from tensorflow.keras import optimizers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
model_n = 28
opt = optimizers.Adam()

In [3]:
# load json and create model
model_path = 'results/NNmodel_{}_with_1.0data.json'.format(model_n)
print(model_path)
weights_path = 'results/NNmodel_{}_with_1.0data_weights.hdf5'.format(model_n)

json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(weights_path)
print("Loaded model from disk")

# evaluate loaded model 

loaded_model.compile(loss='mean_squared_error', optimizer=opt,
                     metrics=['mean_squared_error'])

W0819 18:27:09.825403 139950472767232 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 18:27:09.826366 139950472767232 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 18:27:09.828861 139950472767232 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ze

results/NNmodel_28_with_1.0data.json
Loaded model from disk


In [90]:
Xdev = np.load('data/Xdev.npy')
ydev = np.load('data/ydev.npy')

In [46]:
pred_dev = loaded_model.predict(Xdev)
dev_mse = mean_squared_error(ydev, pred_dev)
dev_mse

0.07905141412700614

## Use permutation to analyze the importance of features

In [119]:
n_example, n_features = Xdev.shape
n_features

994

In [132]:
# a = np.arange(12).reshape((3,4))
# a

In [133]:
# a[:,1] = np.random.permutation(a[:,1])

In [134]:
# a = np.arange(12).reshape((3,4))
# np.random.permutation(a[:,1])
# a

In [135]:
# len(a[0])

In [136]:
# a = np.arange(12).reshape((3,4))

# for i in range(len(a[0])):
#     permut_a = np.zeros(np.shape(a))
#     permut_a[:] = a
#     permut_a[:,i] = np.random.permutation(a[:,i])
#     print(permut_a)

## trial with fake settings

In [105]:
t1 = time.time()

permut_X = Xdev

permut_X[:,990] = np.random.permutation(Xdev[:,990])
pred_permute = loaded_model.predict(permut_X)
mse_permute = mean_squared_error(ydev, pred_permute)
print(mse_permute)

t2 = time.time()
print('used time in {} seconds'.format(t2-t1))

0.08164757472081949
used time in 0.2709469795227051 seconds


In [107]:
randsample = 3
n_features = 5

In [110]:
permutsample = np.zeros((randsample, n_features))
for trying in range(randsample):
    t1 = time.time()
    print('This is sample {}'.format(trying))
    for i in range(n_features):
        permut_X = np.zeros(np.shape(Xdev))
        permut_X[:] = Xdev
        permut_X[:,-i] = np.random.permutation(Xdev[:,-i])
        permut_pred = loaded_model.predict(permut_X)
        permut_mse = mean_squared_error(ydev, permut_pred)
        permutsample[trying, -i] = permut_mse
    t2 = time.time()
    print('Use time in {} seconds'.format(t2-t1))
    print('*' * 20)


This is sample 0
Use time in 1.5444622039794922 seconds
********************
This is sample 1
Use time in 1.551302194595337 seconds
********************
This is sample 2
Use time in 1.545614242553711 seconds
********************


In [111]:
print(permutsample)

[[0.08166467 0.07619916 0.09034454 0.13429679 0.08338546]
 [0.08168226 0.08278596 0.08930428 0.13448791 0.08589154]
 [0.08167277 0.08031207 0.088329   0.17884274 0.08598124]]


In [112]:
# np.save('results/permut_dev_results.npy', permutsample)

In [115]:
## train within the .py file. Now load the data
permutsample = np.load('results/permut_dev_results.npy')
permutsample.shape

(10, 994)

In [121]:
#print mean and standard deviation of error
errperformance = np.zeros((n_features, 2))
for i in range(n_features):
    errperformance[i,0] = np.mean(permutsample[:,i])
    errperformance[i,1] = np.std(permutsample[:,i])
errperformance[:,0] = errperformance[:,0] - dev_mse

print(errperformance)

[[2.15767984e-05 1.07859416e-05]
 [0.00000000e+00 0.00000000e+00]
 [4.38557610e-04 4.62754925e-04]
 ...
 [1.89221886e-02 1.47009780e-02]
 [6.72869469e-02 2.14709661e-02]
 [8.83506795e-03 9.10399258e-03]]


## Complete running the code within .py file and load the results

In [123]:
features = pd.read_csv('encoded_columns.txt', delimiter='\t', header=None)

In [124]:
features.columns = ['feature_name']

In [131]:
features['delta_mse'] = errperformance[:,0]
features['std_mse_change'] = errperformance[:,1]
features = features.sort_values('delta_mse', axis=0, ascending=False)
features

,feature_name,delta_mse,std_mse_change
845,TeamEnabledUsers,1.250310e+00,9.827591e-02
194,CountryCode=US,1.169008e+00,1.223369e-01
913,AU_outlook_04,3.739945e-01,2.779143e-02
890,PaidPhoneSystemSeats,1.978075e-01,1.255389e-02
644,TopParents_CountryCode=MUS,1.658943e-01,1.025886e-01
886,PaidIntuneSeats,1.285023e-01,6.036708e-02
940,AU_powerpoint,1.263648e-01,1.474696e-01
441,SignupLocationInfo_Country=Palestinian Authority,1.171379e-01,2.623458e-02
733,TopParents_Industry=Health Provider,9.764050e-02,1.440689e-02
993,FirstPaidProPlusStartDate,7.492895e-02,8.422917e-03


In [137]:
len(features)

994

## Look at some examples on dev set

In [140]:
outputs_name = [
     'AR_exchange_06',
 'AR_sharepoint_06',
 'AR_skype_06',
 'AR_teams_06',
 'AR_od4b_06',
 'AR_onenote_06',
 'AR_word_06',
 'AR_excel_06',
 'AR_powerpoint_06',
 'AR_outlook_06',
 'AR_eslt_06',
 'AR_officeclient_06'
]

In [141]:
Xdev_show = Xdev[:3]
ydev_show = ydev[:3]
predictions = loaded_model.predict(Xdev_show)
# print('predictions shape:', predictions.shape)
data = np.concatenate((ydev_show,predictions), axis=0)
data_df = pd.DataFrame(data=data, columns=outputs_name)
data_df

,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06
0,0.734375,0.000000,0.000000,0.000000,0.000000,0.000000,0.047222,0.030556,0.002778,0.297222,0.652778,0.297222
1,0.887500,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.012500,0.000000,0.062500,0.887500,0.075000
2,1.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.375000,0.000000
3,0.750255,0.024719,0.004190,0.008206,0.025986,0.002643,0.099981,0.110787,0.017456,0.239693,0.756913,0.251244
4,0.847031,0.023172,-0.000008,0.001345,0.004079,-0.000968,0.040239,0.037580,0.005022,0.090785,0.825802,0.098807
5,1.390356,0.018714,0.004390,-0.002590,-0.009068,-0.003955,0.002893,-0.019134,-0.003620,0.033304,1.296405,0.030096


In [143]:
for i in range(len(ydev_show)):
    print('This is for dev example {}'.format(i))
    
    n = Xdev_show.shape[1]
    predictions = loaded_model.predict(Xdev_show[i].reshape((1, n)))
    
    for j in range(len(ydev_show[i])):
        print('{}_true: {}'.format(outputs_name[j], ydev_show[i][j]))
        print('{}_pred: {}'.format(outputs_name[j], predictions[0][j]))
        print('='*50)
    
    print('*'*20)

This is for dev example 0
AR_exchange_06_true: 0.734375
AR_exchange_06_pred: 0.7502551674842834
AR_sharepoint_06_true: 0.0
AR_sharepoint_06_pred: 0.02471877634525299
AR_skype_06_true: 0.0
AR_skype_06_pred: 0.004190310835838318
AR_teams_06_true: 0.0
AR_teams_06_pred: 0.008206300437450409
AR_od4b_06_true: 0.0
AR_od4b_06_pred: 0.025985538959503174
AR_onenote_06_true: 0.0
AR_onenote_06_pred: 0.002643150743097067
AR_word_06_true: 0.04722222222222222
AR_word_06_pred: 0.0999809205532074
AR_excel_06_true: 0.03055555555555556
AR_excel_06_pred: 0.11078701913356781
AR_powerpoint_06_true: 0.0027777777777777783
AR_powerpoint_06_pred: 0.017455939203500748
AR_outlook_06_true: 0.2972222222222222
AR_outlook_06_pred: 0.2396925687789917
AR_eslt_06_true: 0.6527777777777778
AR_eslt_06_pred: 0.7569128274917603
AR_officeclient_06_true: 0.2972222222222222
AR_officeclient_06_pred: 0.2512442171573639
********************
This is for dev example 1
AR_exchange_06_true: 0.8875
AR_exchange_06_pred: 0.84703147411346